In [1]:
import pulp

# Definir el problema
prob = pulp.LpProblem("Problema_de_Transporte", pulp.LpMinimize)

# Definir las variables de decisión
rutas = [(i, j) for i in ['A', 'B', 'C'] for j in [1, 2, 3, 4]]
x = pulp.LpVariable.dicts("ruta", rutas, lowBound=0, cat='Continuous')

# Definir los costos de transporte
costos = {
    ('A', 1): 1000000, ('A', 2): 1200000, ('A', 3): 1500000, ('A', 4): 1300000,
    ('B', 1): 1100000, ('B', 2): 1000000, ('B', 3): 1400000, ('B', 4): 1200000,
    ('C', 1): 1300000, ('C', 2): 1200000, ('C', 3): 700000, ('C', 4): 900000
}

# Función objetivo
prob += pulp.lpSum([x[i, j] * costos[i, j] for (i, j) in rutas]), "Costo_Total"

# Restricciones de capacidad de las fábricas
capacidades = {'A': 500, 'B': 600, 'C': 400}
for i in ['A', 'B', 'C']:
    prob += pulp.lpSum([x[i, j] for j in [1, 2, 3, 4]]) <= capacidades[i], f"Capacidad_{i}"

# Restricciones de demanda de los centros de distribución
demandas = {1: 300, 2: 400, 3: 500, 4: 300}
for j in [1, 2, 3, 4]:
    prob += pulp.lpSum([x[i, j] for i in ['A', 'B', 'C']]) >= demandas[j], f"Demanda_{j}"

# Restricciones de número de camiones
capacidad_camion = 100
max_camiones = {'A': 5, 'B': 6, 'C': 4}
for i in ['A', 'B', 'C']:
    prob += pulp.lpSum([x[i, j] for j in [1, 2, 3, 4]]) <= max_camiones[i] * capacidad_camion, f"Max_Camiones_{i}"

# Resolver el problema
prob.solve()

# Imprimir los resultados
print("Estado:", pulp.LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Costo Total =", pulp.value(prob.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/y7/xnq2gn1x2cs79dhd19wnj9wm0000gn/T/43190975cfa846c7873a087fd4d351e1-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/y7/xnq2gn1x2cs79dhd19wnj9wm0000gn/T/43190975cfa846c7873a087fd4d351e1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 64 RHS
At line 75 BOUNDS
At line 76 ENDATA
Problem MODEL has 10 rows, 12 columns and 36 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (-3) rows, 12 (0) columns and 24 (-12) elements
0  Obj 0 Primal inf 1500 (4)
8  Obj 1.5e+09
Optimal - objective value 1.5e+09
After Postsolve, objective 1.5e+09, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1500000000 - 8 iterations time 0.002, Presolv

In [3]:
# Imprimir los resultados
if prob.status == pulp.LpStatusOptimal:
    costo_minimo = pulp.value(prob.objective)
    print(f"El costo total de transporte mínimo es: {costo_minimo}")
    for i in ['A', 'B', 'C']:
        for j in [1, 2, 3, 4]:
            cantidad = x[(i, j)].varValue
            if cantidad > 0:
                camiones = cantidad / capacidad_camion
                print(f"Se deben transportar {cantidad} unidades desde la Fábrica {i} hasta el Centro {j} en {camiones} camiones.")
else:
    print("El problema no tiene solución óptima.")

El costo total de transporte mínimo es: 1500000000.0
Se deben transportar 300.0 unidades desde la Fábrica A hasta el Centro 1 en 3.0 camiones.
Se deben transportar 100.0 unidades desde la Fábrica A hasta el Centro 3 en 1.0 camiones.
Se deben transportar 100.0 unidades desde la Fábrica A hasta el Centro 4 en 1.0 camiones.
Se deben transportar 400.0 unidades desde la Fábrica B hasta el Centro 2 en 4.0 camiones.
Se deben transportar 200.0 unidades desde la Fábrica B hasta el Centro 4 en 2.0 camiones.
Se deben transportar 400.0 unidades desde la Fábrica C hasta el Centro 3 en 4.0 camiones.
